# STAGE 3a: PROCESS - Clean Monthly CSV File

## This notebook is to document the cleaning steps for each monthly trip data csv file
 1. drop rows with null data
 2. drop rows with trip start time greater than trip stop time
 3. drop rows with station ids associated with lat/lng abnomaly
 4. drop rows with station names with "Temp" indicating test trips
 5. drop rows with durations abnomaly (one trip was almost one month)

- Each csv file is run independently from the others. The cleaned csv files are saved in a separate folder /Data/cleaned_csv for later use.
- A variable <font color="red">rows_to_delete </font> is used to keep track of rows deleted from the original csv. The list of deleted rows are saved in /Data/deleted_rows

In [1]:
import pandas as pd

In [2]:
file_list_df = pd.read_csv('file_list_2020.csv', header=None, names= ['filename'])
file_list = file_list_df['filename'].values

In [3]:
dtypes = {'ride_id': 'str', 'rideable_type': 'category', 'start_station_name': 'category', 'start_station_id': 'category', 'end_station_name':'category',
           'end_station_id': 'category', 'member_casual':'category'}

In [4]:
# this is a function to read the trip files, keep in mind I hard coded the dates columns
def read_csv_to_df(filename, dtype):
    df = pd.read_csv('./Data/csv/'+filename, parse_dates=['started_at','ended_at'], dtype = dtype)

    return df

In [5]:
# this file is a random choice
filename = file_list[2]
df = read_csv_to_df(filename, dtypes)

In [6]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

In [7]:
df.shape

(343005, 13)

### Drop rows with null data

In [8]:
rows_to_delete_na = df[df.isnull().any(axis=1)==True].index
len(rows_to_delete_na)

468

In [9]:
df = df[~df.index.isin(rows_to_delete_na)]
df.shape

(342537, 13)

### Drop rows with time end less than time start

In [10]:
rows_to_delete_timenegative = df[(df['ended_at']<df['started_at'])].index
len(rows_to_delete_timenegative)

468

In [11]:
df = df[~df.index.isin(rows_to_delete_timenegative)]
df.shape

(342069, 13)

## Observing lat long consistency per station id

In [12]:
## Check latitude numbers associated with each station id.

In [13]:
# first concatenate start and stop stations to check outliers together
df_lat_eval_start_station = df[['start_station_id','start_lat']].rename(columns={"start_station_id": "station_id", "start_lat": "station_lat"})
df_lat_eval_end_station = df[['end_station_id','end_lat']].rename(columns={"end_station_id": "station_id", "end_lat": "station_lat"})
df_lat_eval = pd.concat([df_lat_eval_start_station,df_lat_eval_end_station],axis=0)
df_lat_eval.reset_index(inplace=True)
df_lat_eval.columns = ['index_orig', 'station_id', 'station_lat']

In [14]:
from scipy import stats
from scipy.stats import zscore

In [15]:
# try to find outliers for latitude number for each station id# delete rows with latitude number that is not consistent with the station id

In [16]:
# the number of samples should be greater than 30 for relevant zscore calculation
c = df_lat_eval.groupby(['station_id'])['station_id'].count()
few_samples_station_ids= c[c<30].index
df_lat_eval.drop(df_lat_eval[df_lat_eval['station_id'].isin(few_samples_station_ids)].index, inplace=True)

In [17]:
# calculate z score using groupby station_id

In [18]:
df_zscore_lat = df_lat_eval.groupby(['station_id'])['station_lat'].transform(lambda x : zscore(x,ddof=1))
df_zscore_lat.name = 'zscore'

In [19]:
df_lat_eval_zscore = pd.concat([df_lat_eval,df_zscore_lat],axis=1)
rows_to_delete_lat = df_lat_eval_zscore[(df_lat_eval_zscore['zscore']<-3)|(df_lat_eval_zscore['zscore']>3)]
rows_to_delete_list_lat=list(rows_to_delete_lat['index_orig'])

In [20]:
rows_to_delete_lat

,index_orig,station_id,station_lat,zscore
250133,250823,120,41.853005,-10.882343
253043,253742,120,41.853005,-10.882343
272283,273026,120,41.853005,-10.882343
322418,323286,120,41.853005,-10.882343
370176,28210,120,41.853005,-10.882343
534399,192874,120,41.853005,-10.882343
584201,242803,120,41.853005,-10.882343
595117,253747,120,41.853005,-10.882343
664486,323285,120,41.853005,-10.882343


In [21]:
# use the same code to run on longitude

In [22]:
df_lng_eval_start_station = df[['start_station_id','start_lng']].rename(columns={"start_station_id": "station_id", "start_lng": "station_lng"})
df_lng_eval_end_station = df[['end_station_id','end_lng']].rename(columns={"end_station_id": "station_id", "end_lng": "station_lng"})
df_lng_eval_start_station['start_end'] = 'start'
df_lng_eval_end_station['start_end'] = 'end'
df_lng_eval = pd.concat([df_lng_eval_start_station,df_lng_eval_end_station],axis=0)
df_lng_eval.reset_index(inplace=True)
df_lng_eval.columns = ['index_orig', 'station_id', 'station_lng', 'start_end']

c = df_lng_eval.groupby(['station_id'])['station_id'].count()
few_samples_station_ids= c[c<30].index
few_samples_station_ids # exclude these station ids
df_lng_eval.drop(df_lng_eval[df_lng_eval['station_id'].isin(few_samples_station_ids)].index, inplace=True)

df_zscore_lng = df_lng_eval.groupby(['station_id'])['station_lng'].transform(lambda x : zscore(x,ddof=1))
df_zscore_lng.name = 'zscore'
df_lng_eval_zscore = pd.concat([df_lng_eval,df_zscore_lng],axis=1)
rows_to_delete_lng = df_lng_eval_zscore[(df_lng_eval_zscore['zscore']<-3)|(df_lng_eval_zscore['zscore']>3)]

rows_to_delete_list_lng=list(rows_to_delete_lng['index_orig'])

In [23]:
rows_to_delete_list_lat

[250823, 253742, 273026, 323286, 28210, 192874, 242803, 253747, 323285]

In [24]:
rows_to_delete_list_lng

[250823, 253742, 273026, 323286, 28210, 192874, 242803, 253747, 323285]

In [25]:
## it turns out the lat, long outliers are from the same station 120
# further observation shows that the station name of the outlier has (Temp) in it

In [26]:
df[df.index.isin(rows_to_delete_list_lat)]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
28210,CB622F3F3943D307,docked_bike,2020-06-03 18:47:09,2020-06-03 18:53:59,Wabash Ave & Roosevelt Rd,59,Wentworth Ave & Cermak Rd (Temp),120,41.867227,-87.625961,41.853005,-87.630578,member
192874,0D1FED7558DAEDA9,docked_bike,2020-06-03 16:52:32,2020-06-03 17:11:55,Federal St & Polk St,41,Wentworth Ave & Cermak Rd (Temp),120,41.872077,-87.629543,41.853005,-87.630578,casual
242803,F4B0E2B5B8D3A809,docked_bike,2020-06-03 16:50:24,2020-06-03 17:12:04,Federal St & Polk St,41,Wentworth Ave & Cermak Rd (Temp),120,41.872077,-87.629543,41.853005,-87.630578,casual
250823,E339E63520B6B694,docked_bike,2020-06-03 19:29:09,2020-06-03 19:38:03,Wentworth Ave & Cermak Rd (Temp),120,Wabash Ave & Roosevelt Rd,59,41.853005,-87.630578,41.867227,-87.625961,casual
253742,5B8C6C6643F7B800,docked_bike,2020-06-03 10:28:39,2020-06-03 10:52:04,Wentworth Ave & Cermak Rd (Temp),120,Wabash Ave & Roosevelt Rd,59,41.853005,-87.630578,41.867227,-87.625961,member
253747,7949EB2C8254A922,docked_bike,2020-06-03 10:58:17,2020-06-03 11:11:25,Wabash Ave & Roosevelt Rd,59,Wentworth Ave & Cermak Rd (Temp),120,41.867227,-87.625961,41.853005,-87.630578,member
273026,D0C9C616E9C73DA0,docked_bike,2020-06-03 06:36:30,2020-06-03 06:49:15,Wentworth Ave & Cermak Rd (Temp),120,Clinton St & Madison St,77,41.853005,-87.630578,41.882242,-87.641066,member
323285,598377F12456DE37,docked_bike,2020-06-03 15:35:40,2020-06-03 15:49:52,Clinton St & Madison St,77,Wentworth Ave & Cermak Rd (Temp),120,41.882242,-87.641066,41.853005,-87.630578,member
323286,32BA9A607CE885C3,docked_bike,2020-06-04 06:39:14,2020-06-04 06:53:09,Wentworth Ave & Cermak Rd (Temp),120,Clinton St & Madison St,77,41.853005,-87.630578,41.882242,-87.641066,member


### It is very possible the station names with (Temp) are for testing only
The rows that contain station name with Temp ought to be deleted

In [27]:
rows_to_delete_temp = list(df[df['start_station_name'].str.contains("Temp")].index)

### Now do the final clean up by deleting rows with lat/lng outlier, and with temp stations

In [28]:
rows_to_drop = list(set(rows_to_delete_list_lat) | set(rows_to_delete_list_lng) | set(rows_to_delete_temp) )
df = df[~df.index.isin(rows_to_drop)]

In [29]:
df.shape

(339983, 13)

### Calculate trip duration and examine outliers

In [30]:
import datetime

In [31]:
duration  = (df['ended_at']-df['started_at']).apply(datetime.timedelta.total_seconds)

In [32]:
duration.name ='duration'

In [33]:
duration_zscore_calc = ((duration - duration.mean())/duration.std())

In [34]:
duration_outliers = duration[(duration_zscore_calc>3) | (duration_zscore_calc<-3)]

In [35]:
duration_outliers.min()/60/60

12.953611111111112

In [36]:
rows_to_delete_duration = duration_outliers.index

In [37]:
duration_outliers.max()/60/60/24

28.660416666666666

In [38]:
df.loc[duration_outliers[duration_outliers==duration_outliers.max()].index]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
162077,BE755E8C79D4E917,docked_bike,2020-06-03 18:13:50,2020-07-02 10:04:50,Wolcott Ave & Polk St,342,Kedzie Ave & Harrison St,433,41.871262,-87.673688,41.8736,-87.704871,member


In [39]:
df.shape

(339983, 13)

In [40]:
df = df[~df.index.isin(rows_to_delete_duration)]
df.shape

(339194, 13)

### Collect all the index of all the deleted rows and output as csv for record

In [41]:
rows_deleted = list(set(rows_to_delete_na) |\
                    set(rows_to_delete_timenegative)|\
                    set(rows_to_delete_duration)|\
                    set(rows_to_delete_list_lat) | \
                    set(rows_to_delete_list_lng) | \
                    set(rows_to_delete_temp) ) 

In [42]:
len(rows_deleted)

3811

In [48]:
file_dest = './data/deleted_rows/row_deleted_for_'+filename

pd.DataFrame(rows_deleted).to_csv(file_dest)

In [51]:
df.to_csv('./data/cleaned_csv/'+filename)